In [ ]:
from fn_cfg import *
import params as cfg
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import joblib

In [ ]:
#dataset = pd.read_csv('/Users/joshuaighalo/Documents/BrainNet/Projects/Workspace/results/laurel place/ml_dataset/eye_status_20s.csv')
#dataset = pd.read_csv('/Users/joshuaighalo/Documents/BrainNet/Projects/Workspace/results/laurel place/ml_dataset/eye_status_30s.csv')
dataset = pd.read_csv('/Users/joshuaighalo/Documents/BrainNet/Projects/Workspace/results/laurel place/ml_dataset/eye_status_10s.csv')
X = dataset.drop('eye_status', 1)
y = dataset['eye_status'].array

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
pca = PCA()
dfx_pca = pca.fit(X_scaled)
explained_variance = pca.explained_variance_ratio_
idx = np.where(explained_variance > 0.05)[0]
print(X.columns.values[idx])
X_scaled = X_scaled[:,idx]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33)

In [ ]:
#SVM
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
rf_params = {
    'C': stats.uniform(0,50),
    "kernel":['linear','poly','rbf','sigmoid'],
    "degree": np.arange(1,7,1)
}
n_iter_search=20
clf = SVC(gamma='scale')
Random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=n_iter_search,cv=3,scoring='accuracy')
Random.fit(X_train, y_train)
print(Random.best_estimator_)
print("Accuracy:"+ str(Random.best_score_))

In [ ]:
Random.best_estimator_.fit(X_train, y_train)

In [ ]:
# save the model to disk
filename = 'best_SVC_model.sav'
joblib.dump(Random.best_estimator_, filename)

# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, y_test)

In [ ]:
#y_pred = Random.best_estimator_.predict(X_train)
y_pred = loaded_model.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
np.sqrt(mse)

In [ ]:
y_pred = loaded_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
np.sqrt(mse)
accuracy_score(y_test, y_pred)

In [ ]:
def averageProcessedEEG(device_version,scan_ID,local_path,fs,line,lowcut,highcut):
    device = importFile.neurocatch()
    fileObjects = device.init(device_version,scan_ID,local_path,dispIMG=False)
    rawEEG = fileObjects[0]
    rawEOG = fileObjects[1]
    filtering = filters()
    adaptiveFilterOutput = filtering.adaptive(rawEEG,rawEOG)
    notchFilterOutput = filtering.notch(adaptiveFilterOutput,line,fs)
    bandPassFilterOutput = filtering.butterBandPass(notchFilterOutput,lowcut,highcut,fs)[:,1]
    windowEEG = slidingWindow(bandPassFilterOutput,5000,5000)
    return windowEEG


filename = "No_Threshold.csv"
device_version = 1.0
lowcut = cfg.highPass
highcut = cfg.lowPass
sfreq = cfg.fs
localPath_scans = "/Users/joshuaighalo/Documents/BrainNet/Projects/Workspace/results/laurel place/quality scans/"
file_dir = localPath_scans+filename
quality_scans = pd.read_csv(file_dir)
base_ND = [x for x in (quality_scans['Baseline No Dementia'].to_numpy()) if str(x) != 'nan']


scans_windowEEG = []
for i in range(len(base_ND)):
    scan_ID = base_ND[i]
    windowEEG = averageProcessedEEG(device_version,scan_ID,localPath_scans,sfreq,line,lowcut,highcut)
    scans_windowEEG.append(windowEEG)

In [ ]:
def featureExtraction(data):
    filtering = filters()
    delta_rhythms = filtering.butterBandPass(data,0.5,4,fs)
    theta_rhythms = filtering.butterBandPass(data,4,8,fs)
    alpha_rhythms = filtering.butterBandPass(data,8,13,fs)
    beta_rhythms = filtering.butterBandPass(data,13,32,fs)
    gamma_rhythms = filtering.butterBandPass(data,32,79,fs)

    std_delta_rhythms = np.std(delta_rhythms,axis=0)
    std_theta_rhythms = np.std(theta_rhythms,axis=0)
    std_alpha_rhythms = np.std(alpha_rhythms,axis=0)
    std_beta_rhythms = np.std(beta_rhythms,axis=0)
    std_gamma_rhythms = np.std(gamma_rhythms,axis=0)

    rms_delta_rhythms = np.sqrt(np.mean(delta_rhythms**2))
    rms_theta_rhythms = np.sqrt(np.mean(theta_rhythms**2))
    rms_alpha_rhythms = np.sqrt(np.mean(alpha_rhythms**2))
    rms_beta_rhythms = np.sqrt(np.mean(beta_rhythms**2))
    rms_gamma_rhythms = np.sqrt(np.mean(gamma_rhythms**2))

    var_delta_rhythms = np.var(delta_rhythms,axis=0)
    var_theta_rhythms = np.var(theta_rhythms,axis=0)
    var_alpha_rhythms = np.var(alpha_rhythms,axis=0)
    var_beta_rhythms = np.var(beta_rhythms,axis=0)
    var_gamma_rhythms = np.var(gamma_rhythms,axis=0)

    kurtosis_delta_rhythms = stats.kurtosis(delta_rhythms,axis=0)
    kurtosis_theta_rhythms = stats.kurtosis(theta_rhythms,axis=0)
    kurtosis_alpha_rhythms = stats.kurtosis(alpha_rhythms,axis=0)
    kurtosis_beta_rhythms = stats.kurtosis(beta_rhythms,axis=0)
    kurtosis_gamma_rhythms = stats.kurtosis(gamma_rhythms,axis=0)
    #features = np.vstack((std_delta_rhythms,std_theta_rhythms,std_alpha_rhythms,std_beta_rhythms,std_gamma_rhythms,rms_delta_rhythms,rms_theta_rhythms,rms_alpha_rhythms,rms_beta_rhythms,rms_gamma_rhythms,var_delta_rhythms,var_theta_rhythms,var_alpha_rhythms,var_beta_rhythms,var_gamma_rhythms,kurtosis_delta_rhythms,kurtosis_theta_rhythms,kurtosis_alpha_rhythms,kurtosis_beta_rhythms,kurtosis_gamma_rhythms)).T
    features = [std_delta_rhythms,std_theta_rhythms,std_alpha_rhythms,std_beta_rhythms,std_gamma_rhythms,rms_delta_rhythms,rms_theta_rhythms,rms_alpha_rhythms,rms_beta_rhythms,rms_gamma_rhythms,var_delta_rhythms,var_theta_rhythms,var_alpha_rhythms,var_beta_rhythms,var_gamma_rhythms,kurtosis_delta_rhythms,kurtosis_theta_rhythms,kurtosis_alpha_rhythms,kurtosis_beta_rhythms,kurtosis_gamma_rhythms]
    return features


features = []
for i in range(len(allEEG_EO)):
    features.append(featureExtraction(allEEG_EO[i]))